In [1]:
from openai import OpenAI
import time
import numpy as np

In [2]:
with open('openai_api_key.py') as script:
    exec(script.readline())

client = OpenAI(
  api_key=openai_api_key,
)

# Sunk Cost Fallacy
### Using OpenAI Playground - Complete API with gpt-3.5-turbo-instruct

Prompt:

*Assume that you have spent `$90/$250` for a ticket to a theater performance. Several weeks later you buy a `$30` ticket to a rock concert. You think you will enjoy the rock concert more than the theater performance. As you are putting your just-purchased rock concert ticket in your wallet, you notice that both events are scheduled for the same evening. The tickets are non-transferable, nor can they be exchanged. You can use only one of the tickets and not the other. Which ticket will you use?*

*Answer: I would use the ticket to the*

---

From:

- Soman, D. 2001. The mental accounting of sunk time costs: why time is not like money. Journal of Behavioral Decision Making 14(3): 169–185.

- Arkes, H.R., and C. Blumer. 1985. The psychology of sunk cost. Organizational Behavior and Human
Decision Processes 35(1): 124–140.

(both Experiment 1)

---

The OpenAI Complete API Playground allows you to see the probabilities of a token being generated. This allows you to see how likely the model is to choose an option. You can also change the parameters and the amount of sunk cost to see how the probabilities change:

| Parameters |                   | `$30 vs. $90`       |                      | `$30 vs. $250`         |                      |
|---------|------------|----------------------|----------------------|----------------------|----------------------|
|  Temperature | Top P       | Rock Concert         | Theater Performance  | Rock Concert         | Theater Performance  |
| 0.5 | 1                 | 97.34%               | 2.31%                | 91.95%               | 7.67%                |
| 1 | 1                  | 97.35%               | 2.30%                | 91.95%               | 7.67%                |
| 1.5 | 1                 | 97.35%               | 2.31%                | 91.94%               | 7.67%                |
| 1 | 0.5                 | 97.34%               | 2.31%                | 92.55%               | 7.07%                |

Model: gpt-3.5-turbo-instruct

Changing the parameters `Temperature` and `Top P` does not change the probabilities. But changing the amount of sunk costs from `$90` to `$250` slightly increases the probability of choosing the theater performance by about 5 percentage points.

In conclusion, although the model (gpt-3.5-turbo-instruct) isn't significantly affected by sunk cost bias, a slight impact is still observed.

---

### Using gpt-4-1106-preview ("GPT-4 Turbo")

Result: The model did not choose the theater performance once. Therefore, GPT-4 Turbo does not seem to be subject to the sunk cost bias.

In [3]:
def sunk_cost_experiment(model, prompt, instruction, n, temperature=1, max_tokens=2):
    answers = []
    for _ in range(n):
        response = client.chat.completions.create(
            model=model,  
            messages=[
                {"role": "system", "content": instruction},

                {"role": "user", "content": prompt}
            ],
            max_tokens=max_tokens,
            temperature=temperature
        )

        # Store the answer in the list
        answer = response.choices[0].message.content
        answers.append(answer.strip())

        # Add delay before the next API call
        time.sleep(0.1)  
     
    return answers

In [5]:
# gpt-4-1106-preview (GPT-4 Turbo) is more powerful than GPT-4 and offered at a lower price
model = 'gpt-4-1106-preview'

instruction = 'Complete the answer with two words.'

prompt = """Assume that you have spent $250 for a ticket to a theater performance. \
Several weeks later you buy a $30 ticket to a rock concert. You think you \
will enjoy the rock concert more than the theater performance. As you are \
putting your just-purchased rock concert ticket in your wallet, you notice \
that both events are scheduled for the same evening. The tickets are \
non-transferable, nor can they be exchanged. You can use only one of the \
tickets and not the other. Which ticket will you use?

Answer: I would use the ticket to the"""

In [12]:
%%time
answers = sunk_cost_experiment(model, prompt, instruction, n=100)

Wall time: 50.5 s


In [13]:
answers.count('rock concert') / len(answers)

1.0

In [14]:
answers.count('theater performance') / len(answers)

0.0

---

### Using gpt-3.5-turbo-1106

Result: Small sunk cost bias.

In [26]:
model = 'gpt-3.5-turbo-1106'

In [27]:
%%time
answers = sunk_cost_experiment(model, prompt, instruction, n=100)

Wall time: 60 s


In [37]:
(answers.count('rock concert') 
 + answers.count('concert.')
 + answers.count('concert')
 + answers.count('concert performance')) / len(answers)

0.95

In [31]:
answers.count('theater') / len(answers)

0.05

---
## Experiment 2
From
- Thaler, Richard (2015), "Misbehaving"

In [45]:
prompt1="""Suppose you bought a case of good Bordeaux in the futures \
market for $20 a bottle. The wine now sells at auction for about $75. \
You have decided to drink a bottle. Which of the following best captures \
your feeling of the cost to you of drinking the bottle?

A: $0. I alreadey paid for it.
B: $20, what I paid for.
C: $20, plus interest.
D: $75, what I could get if I sold the bottle.
E: -$55, I get to drink a bottle that is worth $75 that I only paid \
$20 for so I save money by drinking the bottle."""

instruction = 'Please answer by only giving the letter of the answer option A, B, C, D or E.'

model = 'gpt-3.5-turbo-1106'

In [15]:
%%time
answers = sunk_cost_experiment(model, prompt1, instruction, n=100, max_tokens=1)

Wall time: 52 s


In [17]:
answers.count("A") / len(answers)

0.01

In [18]:
answers.count("B") / len(answers)

0.05

In [19]:
answers.count("C") / len(answers)

0.0

In [20]:
answers.count("D") / len(answers)

0.0

In [21]:
answers.count("E") / len(answers)

0.94

Results from the book:
- A (30%)
- B (18%)
- C (7%)
- D (20%)
- E (25%)

(Fun fact) Thaler writes: *"When we included option (e), which we found greatly amusing, we were not sure anyone would select it. We wondered wether there were really people who are so sophisticated in their use of mental accounting that they can consider the drinking of an expensive bottle of wine as an act that saves them money. But many people took that option seriously, and over half of the resondents said that drinking the bottle was either free or saved them money."*

Changing the order of the answers to see if anything changes:

In [46]:
prompt2="""Suppose you bought a case of good Bordeaux in the futures \
market for $20 a bottle. The wine now sells at auction for about $75. \
You have decided to drink a bottle. Which of the following best captures \
your feeling of the cost to you of drinking the bottle?

A: $75, what I could get if I sold the bottle.
B: -$55, I get to drink a bottle that is worth $75 that I only paid \
$20 for so I save money by drinking the bottle.
C: $0. I alreadey paid for it.
D: $20, what I paid for.
E: $20, plus interest."""

instruction = 'Please answer by only giving the letter of the answer option A, B, C, D or E.'

model = 'gpt-3.5-turbo-1106'

In [23]:
%%time
answers = sunk_cost_experiment(model, prompt2, instruction, n=100, max_tokens=1)

Wall time: 1min 5s


In [24]:
answers.count("A") / len(answers)

0.04

In [25]:
answers.count("B") / len(answers)

0.64

In [26]:
answers.count("C") / len(answers)

0.02

In [27]:
answers.count("D") / len(answers)

0.3

In [28]:
answers.count("E") / len(answers)

0.0

Indeed some changes, but still the "sophisticated" option favoured.

### With GPT-4 Turbo

In [47]:
model = 'gpt-4-1106-preview'

In [48]:
%%time
answers = sunk_cost_experiment(model, prompt1, instruction, n=100, max_tokens=1)

Wall time: 44.6 s


In [49]:
answers.count("A") / len(answers)

0.0

In [50]:
answers.count("B") / len(answers)

0.0

In [51]:
answers.count("C") / len(answers)

0.0

In [52]:
answers.count("D") / len(answers)

1.0

In [53]:
answers.count("E") / len(answers)

0.0

In [54]:
%%time
answers = sunk_cost_experiment(model, prompt2, instruction, n=100, max_tokens=1)

Wall time: 49.7 s


In [55]:
answers.count("A") / len(answers)

1.0

In [56]:
answers.count("B") / len(answers)

0.0

In [57]:
answers.count("C") / len(answers)

0.0

In [58]:
answers.count("D") / len(answers)

0.0

In [59]:
answers.count("E") / len(answers)

0.0

GPT-4 Turbo consistently selects answer "$75, what I could get if I sold the bottle.", which is the correct answer according to economic theory, but not what humans would choose.